## FIRST ATTEPMT

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the data
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Drop unnecessary columns
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Bin rare categorical data as "Other" for columns with high cardinality
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(
    lambda x: x if x in ["T3", "T4", "T6", "T5", "T19", "T8", "T7"] else "Other")
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(
    lambda x: x if x in ["C1000", "C2000", "C1200", "C3000", "C2100"] else "Other")

# One-hot encode categorical data
application_df = pd.get_dummies(application_df)

# Split the preprocessed data into features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [2]:
# Create a new neural network model
nn_optimized = tf.keras.models.Sequential()

# First hidden layer with more neurons and a different activation function
nn_optimized.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=X_train.shape[1]))

# Second hidden layer with increased neurons
nn_optimized.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer added for more complexity
nn_optimized.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
# Compile the model
nn_optimized.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = nn_optimized.fit(X_train_scaled, y_train, epochs=150)


Epoch 1/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 394us/step - accuracy: 0.7146 - loss: 0.5837
Epoch 2/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - accuracy: 0.7310 - loss: 0.5531
Epoch 3/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - accuracy: 0.7320 - loss: 0.5511
Epoch 4/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.7261 - loss: 0.5562
Epoch 5/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7332 - loss: 0.5513
Epoch 6/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - accuracy: 0.7280 - loss: 0.5516
Epoch 7/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - accuracy: 0.7332 - loss: 0.5457
Epoch 8/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7310 - loss: 0.5471
Epoch 9/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - accuracy: 0.7328 - loss: 0.5476
Epoch 10/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.7375 - loss: 0.5420
Epoch 11/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7335 - loss: 0.5431
Epoch 12/150
858/85

In [4]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - 521us/step - accuracy: 0.7278 - loss: 0.6023
Loss: 0.6023204326629639, Accuracy: 0.7278425693511963


## SECOND ATTEMPT

In [5]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN', 'NAME'])

application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

classification_types_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index
for cls in classification_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

application_df = pd.get_dummies(application_df)

X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_optimized = tf.keras.models.Sequential()

nn_optimized.add(tf.keras.layers.Dense(units=128, activation='tanh', input_dim=X_train_scaled.shape[1]))

nn_optimized.add(tf.keras.layers.Dense(units=64, activation='tanh'))

nn_optimized.add(tf.keras.layers.Dense(units=32, activation='tanh'))
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
nn_optimized.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = nn_optimized.fit(X_train_scaled, y_train, epochs=150, verbose=1)
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/150


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 457us/step - accuracy: 0.7215 - loss: 0.5712
Epoch 2/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.7293 - loss: 0.5552
Epoch 3/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.7315 - loss: 0.5515
Epoch 4/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step - accuracy: 0.7318 - loss: 0.5477
Epoch 5/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.7313 - loss: 0.5490
Epoch 6/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.7321 - loss: 0.5456
Epoch 7/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7384 - loss: 0.5395
Epoch 8/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - accuracy: 0.7335 - loss: 0.5449
Epoch 9/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7363 - loss: 0.5394
Epoch 10/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7399 - loss: 0.5420
Epoch 11/150
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.7424 - loss: 0.5356
Epoch 12/150
858/858 ━━━━━━━━━━

## THRID ATTEMPT

In [7]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=['EIN', 'NAME'])

application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

classification_types_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index
for cls in classification_types_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

application_df = pd.get_dummies(application_df)

X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_optimized = tf.keras.models.Sequential()
nn_optimized.add(tf.keras.layers.Dense(units=256, activation='selu', input_dim=X_train_scaled.shape[1]))
nn_optimized.add(tf.keras.layers.Dropout(0.2))
nn_optimized.add(tf.keras.layers.Dense(units=128, activation='selu'))
nn_optimized.add(tf.keras.layers.Dropout(0.2))
nn_optimized.add(tf.keras.layers.Dense(units=64, activation='selu'))
nn_optimized.add(tf.keras.layers.Dropout(0.2))
nn_optimized.add(tf.keras.layers.Dense(units=32, activation='selu'))
nn_optimized.add(tf.keras.layers.Dense(units=16, activation='selu'))
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn_optimized.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = nn_optimized.fit(X_train_scaled, y_train, epochs=200, verbose=1, batch_size=64)

model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

nn_optimized.save("AlphabetSoupCharity_Optimization.h5")

Epoch 1/200


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6915 - loss: 0.6182
Epoch 2/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7182 - loss: 0.5759
Epoch 3/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7163 - loss: 0.5740
Epoch 4/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7277 - loss: 0.5613
Epoch 5/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7274 - loss: 0.5600
Epoch 6/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7231 - loss: 0.5633
Epoch 7/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7315 - loss: 0.5567
Epoch 8/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7250 - loss: 0.5610
Epoch 9/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7285 - loss: 0.5576
Epoch 10/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7287 - loss: 0.5561
Epoch 11/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7271 - loss: 0.5575
Epoch 12/200
429/429 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Loss: 0.569193422794342, Accuracy: 0.7268221378326416
